# Imports

In [1]:
# Graph & Network Libraries
import networkx as nx
import igraph as ig

# Data Handling
import pandas as pd
import numpy as np
import random
import glob
import os
import re
from collections import deque, defaultdict
from itertools import combinations

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *

# Statistical & Utility Libraries
from scipy.stats import zscore
from tqdm import tqdm
import time

3. (20 pts) The double-edge swap can also be used to insert a speciﬁc amount of randomness into a network. In this way, we can use it to deﬁne a parametric network model that contains a speciﬁed amount of “real” structure, which we can get from an empirical network. Let $r$ be the number of double-edge swaps we have applied to some input graph $G$.

Using the UC Berkeley social network from the FB100 data set, design and carry out a numerical experiment to answer the following question: as a function of $r$, how does the clustering coeﬃcient $C$ and mean path length $\langle \ell \rangle$ relax onto those of the corresponding conﬁguration model? As references, overlay in your plots horizontal lines for the $C$ and $\langle \ell \rangle$ when you have applied $r = 20m$ swaps (which will be the values expected under the conﬁguration model). Comment on how “random” Berkeley’s social network was to begin with, in what ways, and
on the rate at which randomization destroys the empirical patterns.

In [2]:
# Load the Berkeley social network
uc_berkeley_filepath = "../hw1/facebook100txt/Berkeley13.txt"


def read_graph(filepath):
    """Reads the UC Berkeley social network from FB100 dataset."""
    G = nx.read_edgelist(filepath, nodetype=int, comments='id')
    return G


berkeley_graph = read_graph(uc_berkeley_filepath)

# print(
#     f"Nodes: {berkeley_graph.number_of_nodes()}, Edges: {berkeley_graph.number_of_edges()}"
# )

In [3]:
""" 
Please See the rest of the code in the Python file in 

`hw/hw2/prob_3.py`
"""

' \nPlease See the rest of the code in the Python file in \n\n`hw/hw2/prob_3.py`\n'

4. (30 pts) In network data science, the conﬁguration model serves as the key null model for deciding whether some network measure x = f (G) is big or small or typical or unusual. Visit the Index of Complex Networks (ICON) at icon.colorado.edu and obtain your choice of

- 1 online / social network
- 1 food web / biological network, and
- 1 connectome / biological network.

Treating them as simple graphs, design and carry out a numerical experiment to answer the following question for each network: to what degree can the network’s (i) clustering coeﬃcient $C$ and (ii) mean path length $\langle \ell \rangle$ be explained by its degree structure? Display your results using plots showing the null distributions of these statistics under the conﬁguration model, and overlay on that distribution a vertical line showing the empirical value for that network. Discuss (i) whether the empirical values are big, small, typical, or unusual, (ii) what that
conclusion implies about the structure of these networks, and (iii) what hypotheses it suggests about the underlying data generating process that produced these networks.

Hint: A good null distribution requires around 1000 conﬁguration model random graphs.
Applying a double-edge swap operation $r = 20m$ times, starting with the empirical graph
at $r = 0$, is suﬃcient to generate a single corresponding conﬁguration model random graph. The bigger the graphs you choose, the longer your compute times will be.

In [4]:
zebra_filepath = "files/moreno_zebra/out.moreno_zebra_zebra"
food_web_filepath = "files/fw/out.maayan-foodweb"
connectome_filepath = "files/con/rhesus_brain_2.graphml"

In [5]:
def read_out_network_data(filepath, type = 'directed'):
    """
    Reads network data from edge list files, handling comment lines.

    Parameters:
    filepath (str): Path to the network data file

    Returns:
    networkx.Graph: An undirected NetworkX graph object
    """
    try:
        if type == 'directed':
            # Create a directed graph
            G = nx.DiGraph()
        else:
        # Create an undirected graph
            G = nx.Graph()

        # Read edges, skipping comment lines
        with open(filepath, "r") as f:
            for line in f:
                # Skip comment lines starting with %
                if line.startswith("%"):
                    continue

                # Split line into source and target nodes
                parts = line.strip().split()
                if len(parts) >= 2:  # Ensure we have at least source and target
                    source, target = map(int, parts[:2])
                    G.add_edge(source, target)

        return G

    except Exception as e:
        print(f"Error reading network file: {str(e)}")
        print(f"Make sure the file exists at: {filepath}")
        return None


connectome = nx.read_graphml(connectome_filepath)
food_web = read_out_network_data(food_web_filepath, type = 'directed')
zebra_web = read_out_network_data(zebra_filepath, type = 'undirected')

# plot the zebra network
# nx.draw(zebra_web, with_labels=True)
# plt.show()

In [6]:
def determine_largest_connected_component(G):
    """
    Determines the largest connected component of a network.

    Parameters:
    G (networkx.Graph): A NetworkX graph object

    Returns:
    networkx.Graph: The largest connected component
    """
    if G.is_directed():
        largest_cc = max(nx.strongly_connected_components(G), key=len)
    else:
        largest_cc = max(nx.connected_components(G), key=len)
    return G.subgraph(largest_cc).copy()


connectome_lcc = determine_largest_connected_component(connectome)
food_web_lcc = determine_largest_connected_component(food_web)
zebra_web_lcc = determine_largest_connected_component(zebra_web)

# print("\nLargest Connected Components:")
# print(f"Connectome: {connectome_lcc.number_of_nodes()} nodes, {connectome_lcc.number_of_edges()} edges")
# print(f"Food Web: {food_web_lcc.number_of_nodes()} nodes, {food_web_lcc.number_of_edges()} edges")
# print(f"Zebra Web: {zebra_web_lcc.number_of_nodes()} nodes, {zebra_web_lcc.number_of_edges()} edges")


In [7]:
def compute_C_and_L(G):
    """Computes the clustering coefficient and average shortest path length of a graph."""
    clustering_coeff = nx.average_clustering(G)

    # Check if the graph is directed
    if G.is_directed():
        if nx.is_strongly_connected(
            G
        ):  # For directed graphs, check strong connectivity
            path_length = nx.average_shortest_path_length(G)
        else:
            print("Graph is not strongly connected.")
            path_length = None  # Handle disconnected directed graphs
    else:
        if nx.is_connected(G):  # For undirected graphs
            path_length = nx.average_shortest_path_length(G)
        else:
            print("Graph is not connected.")
            path_length = None  # Handle disconnected undirected graphs

    return clustering_coeff, path_length


connectome_C, connectome_L = compute_C_and_L(connectome_lcc)
food_web_C, food_web_L = compute_C_and_L(food_web_lcc)
zebra_web_C, zebra_web_L = compute_C_and_L(zebra_web_lcc)

print("\nNetwork Analysis Results:")
print(f"Connectome: C={connectome_C:.4f}, L={connectome_L:.4f}")
print(f"Food Web: C={food_web_C:.4f}, L={food_web_L:.4f}")
print(f"Zebra Web: C={zebra_web_C:.4f}, L={zebra_web_L:.4f}")


Network Analysis Results:
Connectome: C=0.9100, L=1.1000
Food Web: C=0.3865, L=2.2597
Zebra Web: C=0.8543, L=1.8617


In [8]:
def generate_configuration_model(G, num_samples=1000):
    """Generates a null distribution for clustering coefficient and path length using the configuration model"""
    degrees = [d for n, d in G.degree()]
    clustering_coeffs = []
    path_lengths = []

    for _ in range(num_samples):
        # Generate a configuration model graph
        random_G = nx.configuration_model(degrees)
        # Convert to simple graph (removes self-loops and parallel edges)
        random_G = nx.Graph(random_G)
        random_G.remove_edges_from(nx.selfloop_edges(random_G))

        # Compute metrics
        C = nx.average_clustering(random_G)
        clustering_coeffs.append(C)

        if nx.is_connected(random_G):
            L = nx.average_shortest_path_length(random_G)
            path_lengths.append(L)

    return np.array(clustering_coeffs), np.array(path_lengths)

connectome_C_null, connectome_L_null = generate_configuration_model(connectome_lcc)
food_C_null, food_L_null = generate_configuration_model(food_web_lcc)
zebra_C_null, zebra_L_null = generate_configuration_model(zebra_web_lcc)

In [9]:
def plot_null_distribution(null_values, empirical_value, title, xlabel):
    """Plots the null distribution with an empirical value marker and legend."""

    # Create DataFrames for null distribution and empirical value
    df_null = pd.DataFrame({xlabel: null_values, "type": "Null Distribution"})
    df_empirical = pd.DataFrame({xlabel: [empirical_value], "type": "Empirical Value"})

    # Combine data for plotting
    df = pd.concat(
        [
            df_null, 
            # df_empirical
        ]
    )

    # Create the plot
    plot = (
        ggplot(df, aes(x=xlabel))
        + geom_histogram(
            aes(y="..density..", fill="type"), bins=30, alpha=0.6, color="black"
        )  # Histogram with fill mapped to 'type'
        + geom_density(aes(color="type"), size=1, linetype = 'dashed')  # Density curve mapped to 'type'
        + geom_vline(
            aes(xintercept=xlabel, color="type"),
            data=df_empirical,
            linetype="dashed",
            size=1,
        )  # Ensure empirical value appears in legend
        + theme_minimal()
        + labs(title=title, x=xlabel, y="Density", fill="Legend", color="Legend")
        + scale_fill_manual(
            values={"Null Distribution": "skyblue", "Empirical Value": "red"}
        )
        + scale_color_manual(
            values={"Null Distribution": "blue", "Empirical Value": "red"}
        )
    )

    return plot

# Recreate plots with improved aesthetics
plot_null_distribution(
    food_C_null, food_web_C, "Food Web Clustering Coefficient", "Clustering Coefficient"
).save("figures/food_clustering.png")
plot_null_distribution(food_L_null, food_web_L, "Food Web Path Length", "Path Length").save("figures/food_path.png")

plot_null_distribution(
    connectome_C_null,
    connectome_C,
    "Connectome Clustering Coefficient",
    "Clustering Coefficient",
).save("figures/connectome_clustering.png")
plot_null_distribution(
    connectome_L_null, connectome_L, "Connectome Path Length", "Path Length"
).save("figures/connectome_path.png")

plot_null_distribution(
    zebra_C_null,
    zebra_web_C,
    "Zebra Web Clustering Coefficient",
    "Clustering Coefficient",
).save("figures/zebra_clustering.png")

plot_null_distribution(
    zebra_L_null, zebra_web_L, "Zebra Web Path Length", "Path Length"
).save("figures/zebra_path.png")

/opt/anaconda3/envs/networks/lib/python3.12/site-packages/plotnine/ggplot.py:615: PlotnineWarning: Saving 6.4 x 4.8 in image.
/opt/anaconda3/envs/networks/lib/python3.12/site-packages/plotnine/ggplot.py:616: PlotnineWarning: Filename: figures/food_clustering.png
/opt/anaconda3/envs/networks/lib/python3.12/site-packages/plotnine/ggplot.py:615: PlotnineWarning: Saving 6.4 x 4.8 in image.
/opt/anaconda3/envs/networks/lib/python3.12/site-packages/plotnine/ggplot.py:616: PlotnineWarning: Filename: figures/food_path.png
/opt/anaconda3/envs/networks/lib/python3.12/site-packages/plotnine/ggplot.py:615: PlotnineWarning: Saving 6.4 x 4.8 in image.
/opt/anaconda3/envs/networks/lib/python3.12/site-packages/plotnine/ggplot.py:616: PlotnineWarning: Filename: figures/connectome_clustering.png
/opt/anaconda3/envs/networks/lib/python3.12/site-packages/plotnine/ggplot.py:615: PlotnineWarning: Saving 6.4 x 4.8 in image.
/opt/anaconda3/envs/networks/lib/python3.12/site-packages/plotnine/ggplot.py:616: Plo

5. (20 pts) Null models can also be used to test hypotheses about individual nodes and their positions within the network. For example, is a high centrality value for node i actually just a function of the network’s degree structure? Here, we use this approach to revisit a classic paper in social network analysis.

The Medici family was a powerful political dynasty and banking family in 15th century Florence. The classic network explanation of their power2 argues that their inﬂuence came from positioning themselves as the most central node within the network of prominent Florentine families (see the network figure below). Visit the Index of Complex Networks and obtain a copy of the Medici network data file, under the “Padgett Florentine families” ICON entry. Conduct the following tests of the Medici structural importance hypothesis. Define the harmonic centrality of a node as

$$
C_{i} = \frac{1}{n-1}\sum_{j=1 ; j \neq i}^{n}\frac{1}{\ell_{ij}}    
$$

where $\ell_{ij}$ is the length of the shortest path from node $i$ to node $j$; if there is no such path, i.e., because $i$ and $j$ are in different components, then we define $\ell_{ij} = \inf$.

In [10]:
def read_medici_network(filepath):
    G = nx.Graph()
    family_names = {}

    with open(filepath, "r") as f:
        for line in f:
            match = re.match(r"(\d+)\s([^,]+),\s(\d+)\s(\d+)\s\[(.*)\]", line.strip())
            if match:
                family_id = int(match.group(1))
                family_name = match.group(2)
                num_connections = int(match.group(3))
                connections = match.group(5).strip()

                family_names[family_id] = family_name

                # Add node to the graph even if it has no connections
                G.add_node(family_id)

                if num_connections > 0 and connections:
                    edges = []
                    for conn in connections.split(")"):
                        if conn.strip():
                            pair = conn.strip(" (").split(", ")
                            connected_family, weight = int(pair[0]), int(pair[1])
                            edges.append((family_id, connected_family, weight))

                    for edge in edges:
                        G.add_edge(edge[0], edge[1], weight=edge[2])

    return G, family_names


# Path to your Medici network data file
medici_filepath = "files/Medici network/medici_network.txt"
medici_network, family_names = read_medici_network(medici_filepath)

# # # Visualizing the network
# plt.figure(figsize=(12, 12))
# pos = nx.spring_layout(medici_network, seed=42)  # Layout the nodes

# # # Draw the nodes, edges, and labels
# nx.draw(
#     medici_network,
#     pos,
#     with_labels=True,
#     node_size=3000,
#     node_color="skyblue",
#     font_size=10,
#     font_weight="bold",
#     edge_color="gray",
# )
# plt.title("Medici Family Network")
# plt.show()

(a) Calculate and report the harmonic centrality of each node in the Medici network, and
comment on where in the corresponding ranking the Medici family appears. Then discuss
the degree to which your findings agree with the network explanation of the Medici’s
power, and what the scores say about how important is second most important family.

In [11]:
def calculate_harmonic_centrality(G):
    """
    Calculates the harmonic centrality of nodes in a graph.

    Parameters:
    G (networkx.Graph): A NetworkX graph object

    Returns:
    dict: A dictionary of harmonic centrality values for each node
    """
    # Compute harmonic centrality using NetworkX's built-in function
    harmonic_centrality = nx.harmonic_centrality(G)

    # Print results for debugging
    # print("Harmonic Centrality:", harmonic_centrality)

    for node in G.nodes():
        # Normalize the harmonic centrality by n-1
        harmonic_centrality[node] = (1 / (len(G.nodes()) - 1)) * harmonic_centrality[node]

    return harmonic_centrality


medici_harmonic_centrality = calculate_harmonic_centrality(medici_network)

# can we make a figure for this?
# Create a DataFrame for plotting
df_harmonic = pd.DataFrame(
    {
        "Node": list(medici_harmonic_centrality.keys()),
        "Harmonic Centrality": list(medici_harmonic_centrality.values()),
    }
)

# transform the harmonic centrality values to 1/(n-1) where n is the number of nodes
# df_harmonic["Harmonic Centrality"] = 1 / (len(df_harmonic["Harmonic Centrality"]) - 1) * df_harmonic["Harmonic Centrality"]

# Add family names
df_harmonic["Family Name"] = df_harmonic["Node"].map(family_names)

# Sort by harmonic centrality
df_harmonic = df_harmonic.sort_values("Harmonic Centrality", ascending=False)

# Print sorted ranking
print(df_harmonic)

# Create the plot
plot_harmonic = (
    ggplot(df_harmonic, aes(x="Family Name", y="Harmonic Centrality", fill="factor(Node)"))
    + geom_bar(stat="identity", color="black")  # Keep border for better visibility
    + theme_minimal()
    + labs(
        title="Harmonic Centrality of Medici Family Network",
        x="Node",
        y="Harmonic Centrality",
    )
    # + scale_fill_viridis_d()
    + theme(legend_position="none", axis_text_x=element_text(angle=45, hjust=1), figure_size=(12, 6))
)

plot_harmonic.save("figures/harmonic_centrality.png")

    Node  Harmonic Centrality   Family Name
8      8             0.633333        Medici
6      6             0.538889      Guadagni
12    12             0.533333       Ridolfi
1      1             0.522222       Albizzi
14    14             0.522222       Strozzi
15    15             0.522222    Tornabuoni
3      3             0.480000      Bischeri
2      2             0.472222     Barbadori
4      4             0.461111    Castellani
10    10             0.452222       Peruzzi
13    13             0.438889      Salviati
0      0             0.394444    Acciaiuoli
7      7             0.357778  Lamberteschi
5      5             0.355556        Ginori
9      9             0.317778         Pazzi
11    11             0.000000         Pucci


/opt/anaconda3/envs/networks/lib/python3.12/site-packages/plotnine/ggplot.py:615: PlotnineWarning: Saving 12 x 6 in image.
/opt/anaconda3/envs/networks/lib/python3.12/site-packages/plotnine/ggplot.py:616: PlotnineWarning: Filename: figures/harmonic_centrality.png


(b) Design and carry out a numerical experiment to answer the question: to what degree can each family’s structural importance be explained as purely a function of the network’s degree sequence $\vec{k}$? Use these results to assess the network explanation of the Medici’s power, and explain how your results answer that question. Comment on which other families are more (or less) important than we expect under the null model.

Hint: If a family’s centrality score is ‘typical’ in the null model, then we conclude that its centrality can be explained entirely by the network’s degree sequence; if it not typical, then we conclude that other factors (beyond degrees) are needed to explain its centrality, e.g., it could be more central than we expect, or less!

Hint: As above, 1000 random graphs should give you a good null distribution, and it
should be sufficient to apply r = 20m double-edge swaps to produce one such graph.
Because you need to look at all the families, a good visualization would arrange the node indices on the x-axis and plot for each the corresponding distribution of $(C_{i}^{null})_{j} - C_{i}^{data}$, where $j$ indexes the random graphs. The degree to which each distribution spans $y = 0$ is then informative.

In [12]:
# Load original Medici network
G = medici_network

# Compute harmonic centrality in original network
data_harmonic_centrality = calculate_harmonic_centrality(G)

# Generate null model distributions
m = G.number_of_edges()
num_samples = 1000
centrality_differences = {node: [] for node in G.nodes()}

for i in range(num_samples):
    # Generate null model while preserving degree sequence
    G_null = G.copy()
    G_null = nx.double_edge_swap(G_null, nswap=20 * m, max_tries=1000)

    # Calculate centrality for null model
    null_centrality = calculate_harmonic_centrality(G_null)

    # Store differences between null model and original data
    for node in G.nodes():
        diff = null_centrality[node] - data_harmonic_centrality[node]
        centrality_differences[node].append(diff)

# Prepare data for visualization
plot_data = []
for node in G.nodes():
    diffs = centrality_differences[node]
    family = family_names[node]
    plot_data.extend([(family, d) for d in diffs])

df_plot = pd.DataFrame(plot_data, columns=["Family", "Centrality Difference"])

# Create violin plot of differences
violin_plot = (
    ggplot(df_plot, aes(x="Family", y="Centrality Difference"))
    + geom_violin(aes(fill="Family"), alpha=0.5)
    + geom_point(aes(color="Family"), position=position_jitter(width=0.01), alpha=0.3, size = 1)
    + geom_hline(yintercept=0, linetype="dashed", color="red")
    + labs(
        title="Distribution of Centrality Differences (Null Model - Original)",
        x="Family Name",
        y="Difference in Harmonic Centrality",
    )
    + theme_minimal()
    + theme(axis_text_x=element_text(rotation=45, hjust=1), figure_size=(12, 6), legend_position="none")
)

violin_plot.save("figures/centrality_differences.png")

/opt/anaconda3/envs/networks/lib/python3.12/site-packages/plotnine/ggplot.py:615: PlotnineWarning: Saving 12 x 6 in image.
/opt/anaconda3/envs/networks/lib/python3.12/site-packages/plotnine/ggplot.py:616: PlotnineWarning: Filename: figures/centrality_differences.png


(c) (10 pts extra credit) Repeat the above experiment but change the null model to one that uses “the wrong” graph space. Specifically, use the stub-matching algorithm to construct stub-labeled loopy multigraphs, which you then “simplify” by removing self-loops and collapsing multiedges. Make the same plot as before, and discuss how your results here differ (if at all) from when you use the correct null model (one that matches the data’s graph space), and what conclusions would change (if any)

In [13]:
def generate_wrong_null_model(G):
    """
    Generate a null model using configuration model that creates a multigraph,
    then simplify it by removing self-loops and collapsing multiple edges.
    """
    # Get degree sequence from original graph
    degree_sequence = [d for _, d in G.degree()]

    # Generate multigraph using configuration model
    G_null = nx.configuration_model(degree_sequence)

    # Convert to simple graph (this step collapses multiple edges and removes self-loops)
    G_null = nx.Graph(G_null)

    return G_null


# Load original Medici network
G = medici_network
degree_sequence = [d for _, d in G.degree()]

# Compute harmonic centrality in original network
data_harmonic_centrality = calculate_harmonic_centrality(G)

# Generate null model distributions
num_samples = 1000
centrality_differences = {node: [] for node in G.nodes()}

for i in range(num_samples):
    # Generate null model using configuration model and simplification
    G_null = generate_wrong_null_model(G)

    # Calculate centrality for null model
    null_centrality = calculate_harmonic_centrality(G_null)

    # Store differences between null model and original data
    for node in G.nodes():
        # Handle case where node might be disconnected in null model
        null_value = null_centrality.get(node, 0)
        diff = null_value - data_harmonic_centrality[node]
        centrality_differences[node].append(diff)


# Prepare data for visualization
plot_data = []
for node in G.nodes():
    diffs = centrality_differences[node]
    family = family_names[node]
    plot_data.extend([(family, d) for d in diffs])

df_plot = pd.DataFrame(plot_data, columns=["Family", "Centrality Difference"])

# Create violin plot of differences
plot = (
    ggplot(df_plot, aes(x="Family", y="Centrality Difference"))
    + geom_violin(aes(fill="Family"), alpha=0.5)
    + geom_point(aes(color="Family"), position=position_jitter(width=0.01), alpha=0.3, size = 1)
    + geom_hline(yintercept=0, linetype="dashed", color="red")
    + labs(
        title="Distribution of Centrality Differences (Stub-Matching Null - Original)",
        x="Family Name",
        y="Difference in Harmonic Centrality",
    )
    + theme_minimal()
    + theme(axis_text_x=element_text(rotation=45, hjust=1), figure_size=(12, 6), legend_position="none")
)

plot.save("figures/centrality_differences_stub_matching.png")

/opt/anaconda3/envs/networks/lib/python3.12/site-packages/plotnine/ggplot.py:615: PlotnineWarning: Saving 12 x 6 in image.
/opt/anaconda3/envs/networks/lib/python3.12/site-packages/plotnine/ggplot.py:616: PlotnineWarning: Filename: figures/centrality_differences_stub_matching.png


6. (20 pts extra credit) In the Watts-Strogatz model, the ﬁrst few rewired edges play an outsized role in collapsing the path-length distribution, as they act like highways for shortest paths to cross the center of the ring. But as we rewire more edges, each highway carries progressively less traﬃc because we’re creating many alternate avenues and the shortest-path traﬃc gets distributed more evenly until, when $p = 1$, every edge should carry roughly an equal fraction
of all shortest paths.

Design and carry out a numerical experiment that shows how the distribution of *betweenness centrality* values relaxes across the network as we rewire progressively more edges. Let $r$ count the number of edges rewired.

In [14]:
def create_ws_with_rewiring_count(n, k, r):
    """
    Create a Watts-Strogatz graph with exactly r edges rewired.

    Parameters:
    n : int - Number of nodes
    k : int - Each node is connected to k nearest neighbors
    r : int - Number of edges to rewire

    Returns:
    G : networkx.Graph - The generated graph
    actual_rewired : int - Actual number of edges rewired
    """
    G = nx.watts_strogatz_graph(n, k, 0)
    edges = list(G.edges())
    nodes = list(G.nodes())

    rewired = 0
    attempts = 0
    max_attempts = r * 10

    # Keep track of rewired edges to prevent rewiring the same edge twice
    rewired_edges = set()

    while rewired < r and attempts < max_attempts:
        if not edges:
            break

        # Choose a random edge that hasn't been rewired yet
        available_edges = [(u, v) for u, v in edges if (u, v) not in rewired_edges]
        if not available_edges:
            break

        u, v = available_edges[np.random.randint(len(available_edges))]

        # Choose new target node, excluding current neighbors and self
        possible_targets = [
            w for w in nodes if w != u and w != v and not G.has_edge(u, w)
        ]

        if possible_targets:
            w = np.random.choice(possible_targets)
            G.remove_edge(u, v)
            G.add_edge(u, w)
            rewired += 1
            rewired_edges.add((u, v))
            rewired_edges.add((v, u))  # Add both directions

        attempts += 1

    return G, rewired


def analyze_betweenness_distribution(n=100, k=4, num_points=10, num_iterations=5):
    """
    Analyze betweenness centrality distribution with multiple iterations per rewiring value.

    Parameters:
    n : int - Number of nodes
    k : int - Each node is connected to k nearest neighbors
    num_points : int - Number of different rewiring values to test
    num_iterations : int - Number of networks to generate per rewiring value
    """
    max_rewiring = (n * k) // 4
    r_values = np.unique(np.logspace(0, np.log10(max_rewiring), num_points).astype(int))

    all_data = []

    for r in r_values:
        for iteration in range(num_iterations):
            G, actual_rewired = create_ws_with_rewiring_count(n, k, r)
            bc = nx.betweenness_centrality(G)

            # Calculate additional network metrics
            avg_path_length = nx.average_shortest_path_length(G)
            clustering_coef = nx.average_clustering(G)

            all_data.extend(
                [
                    {
                        "target_rewiring": r,
                        "actual_rewiring": actual_rewired,
                        "betweenness": bc_value,
                        "iteration": iteration,
                        "avg_path_length": avg_path_length,
                        "clustering_coef": clustering_coef,
                    }
                    for bc_value in bc.values()
                ]
            )

    return pd.DataFrame(all_data)


# Run analysis with multiple iterations
n, k = 100, 4
df = analyze_betweenness_distribution(n, k, num_iterations=5)

# Create an enhanced plot
plot = (
    ggplot(
        df,
        aes(
            x="factor(target_rewiring)", y="betweenness", fill="factor(actual_rewiring)"
        ),
    )
    + geom_violin(alpha=0.5)
    + geom_point(aes(color="factor(actual_rewiring)"), alpha=0.5, size=0.5)
    + theme_minimal()
    + labs(
        title="Betweenness Centrality Distribution in Watts-Strogatz Networks",
        x="Number of Edges Rewired (log scale)",
        y="Betweenness Centrality",
        fill="Actual Rewiring",
    )
    + theme(
        plot_title=element_text(size=12),
        plot_subtitle=element_text(size=10),
        axis_text_x=element_text(angle=45, hjust=1),
        figure_size=(12, 6),
        legend_position='none'
    )
)

plot.save("figures/betweenness_distribution.png")

/opt/anaconda3/envs/networks/lib/python3.12/site-packages/plotnine/ggplot.py:615: PlotnineWarning: Saving 12 x 6 in image.
/opt/anaconda3/envs/networks/lib/python3.12/site-packages/plotnine/ggplot.py:616: PlotnineWarning: Filename: figures/betweenness_distribution.png
